# Predictions on Future Dataset

In [ ]:
predict_df = pd.read_csv('data/future_data_2024.csv')

In [ ]:
df2 = predict_df.copy()
df2['FileName'] = 'test_images/Images/' + df2['FileName']

In [ ]:
print(df2['FileName'].apply(type).unique())

[<class 'str'>]


In [ ]:
df2['FileName'] = df2['FileName'].astype(str)


In [ ]:
print(df2['FileName'].apply(type).unique())


[<class 'str'>]


In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import os

def test_preprocess_image(image):
    """Apply same preprocessing as Training"""
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = image.resize((224, 224))  # Resize the image

    image = np.asarray(image)  # Convert to numpy array

    return image

num_samples = df2.shape[0]
images = np.empty((3128, 224,224,3))

for i, row in df2.iterrows():
    filepath = row['FileName']
    image = Image.open(filepath)
    preprocessed_image = test_preprocess_image(image)
    # Do the predictions or store preprocessed_image
    images[i] = preprocessed_image


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(images)
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

In [ ]:
final_predictions = model_tuned.predict(test_dataset)

196/196 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step


In [ ]:
# Verify shape of output
print(f"Shape of activity predictions: {final_predictions[0].shape}")
print(f"Shape of person predictions: {final_predictions[1].shape}")


Shape of activity predictions: (3128, 40)
Shape of person predictions: (3128, 1)


In [ ]:
# Convert the `Class` back
import numpy as np

# Reverse the category mapping
reverse_category_mapping = {idx: category for category, idx in category_mapping.items()}
reverse_person_mapping = {idx: category for category, idx in person_mapping.items()}

# Convert the predictions to class labels
predicted_indices = np.argmax(final_predictions[0], axis=1)

predicted_person_indices = np.round(final_predictions[1]).astype(int).flatten() # Added .flatten() to ensure 1D array

predicted_labels = [reverse_category_mapping[idx] for idx in predicted_indices]
predicted_person_labels = [reverse_person_mapping[idx] for idx in predicted_person_indices]

In [ ]:
predict_df['Class'] = predicted_labels
predict_df['MoreThanOnePerson'] = predicted_person_labels

In [ ]:
# save
predict_df.to_csv('predictions.csv', index=False)